# Feature Engineering

In machine learning, a feature is an individual measurable attribute of a phenomenon being observed. The extraction of features from raw training data is called *feature engineering*. In other words, feature engineering is a process of transforming raw training data into a *representation* suitable for the application of machine learning algorithms.

This process usually requires a certain degree of domain expertise and can be divided into the following stages [[1]](#ref_1): 
- Brainstorming on features
- Deciding what features to create
- Creating features
- Studying how the features impact model's predictive accuracy
- Iterating if necessary

In [ ]:
%matplotlib inline
import pyspark
from functools import reduce
from pyspark.sql import SparkSession, SQLContext, Window
import pyspark.sql.functions as F
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from pyspark.storagelevel import StorageLevel
from pathlib import Path

matplotlib.style.use('ggplot')

data_dir = './data' #str(Path.home()) + '/data'

In [ ]:
from pyspark import SparkContext, SparkConf

In [ ]:
conf = SparkConf()#.setAppName(appName).setMaster(master)
sc = SparkContext(conf=conf)

## Reading input data

In [ ]:
sqlContext = SQLContext(sc)

telemetry_input_df = sqlContext.read.parquet(data_dir + '/telemetry')
logs_input_df = sqlContext.read.parquet(data_dir + '/logs')

telemetry_input_df.printSchema()
logs_input_df.printSchema()

telemetry_entries_count = telemetry_input_df.count()
log_entries_count = logs_input_df.count()

print('Total:\n\n{0:10d} telemetry entries\n{1:10d} log entries'.format(
    telemetry_entries_count, log_entries_count))

## Data exploration

Let's answer several basic questions about the input data set.

### Q: How many unique machines left a telemetry trace?

In this example, it is assumed that all machines are identical; hence, no explicit asset inventory is provided. In practice, equipment metadata, which can include make, model, manufacturing date, start date of service, location and other technical specifications, often serves as an important source of information for feature engineering.

In [ ]:
distinct_machines_df = telemetry_input_df.select('machineID').distinct()
machine_count = distinct_machines_df.count()
print('{0} machine(s)'.format(machine_count))

### Q: What time interval is covered by the telemetry records?

In [ ]:
telemetry_input_df.select(
    F.min('timestamp').alias('start'), F.max('timestamp').alias('end')
).withColumn(
    'duration (days)',
    F.datediff(F.col('end'), F.col('start'))
).show()

### Q: What types of log events are available?

In [ ]:
logs_input_df.select('level').groupBy('level').agg(
    F.count('level')).show()

Log event types follow the common convention:

* **INFO**: 
non-error entries that, among other things, may contain information about scheduled or unscheduled repair procedures
* **WARNING**: 
non-fatal, but potentially harmful or anomalous conditions
* **CRITICAL**:
unrecoverable conditions requiring an intervention (repair)

### Q: What types of CRITICAL events (failures) have been captured in the logs? 

In [ ]:
failures_df = logs_input_df.where(F.col('level') == 'CRITICAL')

failures_df.select('code').groupBy('code').agg(
    F.count(F.lit(1)).alias('count')
).toPandas().set_index('code').plot.pie(
    subplots=True, figsize=(5, 5), autopct='%1.1f%%', title='Failure types')

plt.show()

### Q: How many failures per machine have occured?

The output is a table showing the distribution of failure counts per machine.

In [ ]:
machine_failures_df = distinct_machines_df.join(
    failures_df, 'machineID', 'left_outer'
).groupBy('machineID').agg(F.count('code').alias('count'))

mf_pandas = machine_failures_df.toPandas()
mf_pandas.columns = ['# Of Machines', 'Failure Count']
mf_pandas.groupby('Failure Count').count()

### Q: What are the properties of a single machine's raw telemetry stream?

#### Descriptive statistics

In [ ]:
# get ID of one of the most "faulty" machines
faultyMachineID = machine_failures_df.orderBy(F.desc('count')).first().machineID

faulty_machine_sequence_df = telemetry_input_df.where(
    telemetry_input_df.machineID == faultyMachineID
).orderBy(telemetry_input_df.timestamp)

faulty_machine_sequence_pdf = faulty_machine_sequence_df.toPandas()
faulty_machine_sequence_pdf.set_index('timestamp', inplace=True)
print('Descriptive statistics of "faulty" machine {}'.format(faultyMachineID))
faulty_machine_sequence_pdf.describe()

As a comparison, here shows the descriptive statistics of a **healthy** machine.

In [ ]:
# get ID of one of the "healthy" machines
healthyMachineID = machine_failures_df.orderBy(F.asc('count')).first().machineID

healthy_machine_sequence_df = telemetry_input_df.where(
    telemetry_input_df.machineID == healthyMachineID
).orderBy(telemetry_input_df.timestamp)

healthy_machine_sequence_pdf = healthy_machine_sequence_df.toPandas()
healthy_machine_sequence_pdf.set_index('timestamp', inplace=True)
print('Descriptive statistics of "healthy" machine {}'.format(healthyMachineID))
healthy_machine_sequence_pdf.describe()

#### Temporal properties

In [ ]:
# Data sample
print('Data sample of "faulty" machine {}'.format(faultyMachineID))
faulty_machine_sequence_pdf.head()

In [ ]:
# Occurence of entries in the "faulty" telemetry stream over time

faulty_machine_sequence_pdf.apply(lambda x: 1, axis=1).plot(
    title='Telemetry stream of "faulty" machine {0}'.format(faultyMachineID), style='.', yticks=[])
plt.show()

In [ ]:
# Telemetry frequency
# ('diffs' are computed by subtracting the previous timestamp
# from the current one for each telemetry entry)

diffs = faulty_machine_sequence_pdf.index.to_series().diff()
mode_interval = diffs.mode()[0] # mode of the time intervals
print('Telemetry frequency: {}'.format(mode_interval))

In [ ]:
# Other, less common, intervals between telemetry entries (minutes)

gaps = diffs[diffs != mode_interval].dt.seconds/60
gaps.hist()
plt.title('telemetry frequency of "faulty" machine {}'.format(faultyMachineID))
plt.xlabel('Time [s]')
plt.ylabel('Count')
plt.show()

This shows that the machine operates in cycles followed by periods of inactivity.

### Understanding cycles/patterns of machine activity 

Many real-world machines operate in cycles. A cycle can be thought of as a temporal period which can describe a certain state of operation of a machine. For example, the operation of a turbo-fan engine on a commercial aircraft can be described succinctly by the cycles: engine operating (aircraft in flight) or engine off (aircraft grounded). In this case the cycles are simply describing engine on and off states but this need not be the case. For a slightly different example, we can imagine the cycles of operation of a printer in an office. We will have cycles describing the printer during a print job and others describing the printer which is on but in stand-by mode (that is not printing). Lastly, we could have machines which operate continuously (think of a machine in a manufacturing plant). It may be possible to extract cycles in this type of operating conditions as well but in the case that this is problematic or not beneficial, a cycle can be defined as an hour of operation for example. In the following paragraph we will explain why it is useful to subdivide our data into cycles in predictive maintenance scenarios.

## Feature creation

### Cycle-level telemetry aggregation

Raw telemetry streams, while essential for real-time monitoring and failure detection, cannot be used directly when building predictive maintenance models. The assumption is that sudden degradation during an operational cycle (for instance, a 2-8 hour flight) is rather unlikely. The common practice is to generate a single aggregate measurement to characterize the health of an asset during each unit of operation, which is commonly expressed in hours, or cycles.

In [ ]:
def aggregate_cycles(df, min_gap = 30):
    """
    Detects cycles and compresses them into 
    aggregate measurements.    
    
    :param DataFrame df: input Spark DataFrame
    :param int min_gap: seconds between cycles (mininum)
    """
    w = Window.partitionBy('machineID').orderBy('timestamp')

    # Difference from the previous record or 0 if this is the first one
    diff = F.coalesce(
        F.unix_timestamp('timestamp') - F.unix_timestamp(
            F.lag('timestamp', 1).over(w)
        ), F.lit(0))

    # 0 if diff <= 30, 1 otherwise
    indicator = (diff > min_gap).cast('integer')

    subgroup = F.sum(indicator).over(w).alias('cycle')

    return df.select("*", subgroup).groupBy('machineID', 'cycle').agg(
        F.max('speed_desired').alias('speed_desired_max'),
        F.avg('speed').alias('speed_avg'),
        F.avg('temperature').alias('temperature_avg'),
        F.max('temperature').alias('temperature_max'),
        F.avg('pressure').alias('pressure_avg'),
        F.max('pressure').alias('pressure_max'),
        F.min('timestamp').alias('cycle_start'),
        F.max('timestamp').alias('cycle_end')
    ).orderBy('cycle')

#### Test on the single-machine data set

In [ ]:
machine_cycles_df = aggregate_cycles(faulty_machine_sequence_df)
machine_cycles_df[['temperature_avg',
             'temperature_max',
             'pressure_avg',
             'pressure_max']].toPandas().plot(subplots=True,
                                              title='Cycle-level measurements of "faulty" machine {}'.format(faultyMachineID))
plt.xlabel('Cycles')
plt.show()

As a comparison, here shows the cycle-level measurements of a **healthy** machine.

In [ ]:
machine_cycles_df = aggregate_cycles(healthy_machine_sequence_df)
machine_cycles_df[['temperature_avg',
             'temperature_max',
             'pressure_avg',
             'pressure_max']].toPandas().plot(subplots=True,
                                              title='Cycle-level measurements of "healthy" machine {}'.format(healthyMachineID))
plt.xlabel('Cycles')
plt.show()

#### Transformation of the entire data set

In [ ]:
cycles_df = aggregate_cycles(telemetry_input_df)
print('Total: {0} cycle(s)'.format(cycles_df.count()))

### Labeling

Machine cycles preceding failure events in the logs can be assigned remaining useful life (RUL) labels based on their position in the sequence. For example, the cycle immediately before a failure is assigned RUL equal 1, and the cycle preceding it is assigned RUL equal 2, and so on.

RUL is translated into multi-class labels as follows:

$$
L_i = \left\{
\begin{array}{ll}
      F_i & rul_i\leq w \\
      \emptyset & \text{otherwise} \\
\end{array} 
\right.
$$

where $F_i$ is the type of the failure which ends the sequence the entry is a part of, $rul_i$ is the remaining useful life assigned to the entry, and $w$ is the future horizon.

In [ ]:
# Calculating failure intervals

w = Window.partitionBy('machineID', 'level').orderBy('timestamp')
# the period between consequtive failures,
# or "since the beginning of time" if no previous failure record is on file
diff = F.coalesce(F.lag('timestamp', 1).over(w), F.to_timestamp(F.lit('2000-01-01 00:00:00')))

failure_intervals_df = (failures_df
                .withColumn('last_failure_timestamp', diff)
                .withColumnRenamed('timestamp', 'failure_timestamp')
                .withColumnRenamed('code', 'upcoming_failure')
                .drop('level'))

#### Failure intervals for the selected machine

In [ ]:
machine_failure_intervals_df = failure_intervals_df.where(
    F.col('machineID') == faultyMachineID)
machine_failure_intervals_df.toPandas()

#### Label assignment for all cycles

In addition, for each sequence of cycles running up to a failure, as well as those not ending in a failure, a unique sequence ID is generated.

In [ ]:
w = 7 # future horizon (cycles)

labeled_cycles_df = (
    cycles_df.join(failure_intervals_df,
                   (cycles_df.machineID == failure_intervals_df.machineID) &
                   (cycles_df.cycle_start >= failure_intervals_df.last_failure_timestamp) &
                   (cycles_df.cycle_end <= failure_intervals_df.failure_timestamp),
                   'left_outer')
    .drop(failure_intervals_df.machineID)
    .drop(cycles_df.cycle_end)
    .drop(failure_intervals_df.last_failure_timestamp)
    .withColumnRenamed('cycle_start', 'timestamp')
    .withColumn(
     'rul',                         
      F.when(F.col('upcoming_failure').isNull(), None).otherwise(
         F.row_number().over(Window.partitionBy('machineID', 'failure_timestamp')
                         .orderBy(F.desc('cycle')))))
    .withColumn(
     'sequenceID',
     F.dense_rank().over(Window.partitionBy('machineID')
                   .orderBy(F.desc('upcoming_failure'), 'failure_timestamp')))
    .drop(failure_intervals_df.failure_timestamp)
    .withColumn('immediate_failure',
                F.when(F.col('rul').isNotNull() & (F.col('rul') < w),
                       F.col('upcoming_failure'))))

#### Failure intervals and sequences for the selected machine

In [ ]:
machine_labeled_cycles_df = labeled_cycles_df.where(
    F.col('machineID') == faultyMachineID
).select('timestamp', 'sequenceID', 'upcoming_failure')

max_sequence_id = machine_labeled_cycles_df.select(
    F.max('sequenceID')).collect()[0][0]
machine_labeled_cycles_df.toPandas().set_index('timestamp').plot(
    style='.', yticks=range(1, max_sequence_id + 1))
plt.ylabel('Sequence')
plt.show()

In [ ]:
# Labeled data set
labeled_cycles_df.printSchema()

### Augmenting data with lag features

To help characterize short-term trends for each machine, it is common practice [[3]](#ref_3) to augment the data set with new features computed over rolling windows.

In [ ]:
# TODO: compute more rolling features
# (e.g., linear regression, standard deviation)

lookback = 5
w = Window.partitionBy(
    'machineID', 'sequenceID'
).rowsBetween(-lookback, Window.currentRow).orderBy('cycle')

# computing rolling averages for these cycle-level features
input_columns = ['temperature_avg', 'temperature_max', 'pressure_avg', 'pressure_max']

# adding rolling features and eliminating the entries in the beginning of each
# sequence which weren't computed using a sufficient number of previous values
augmented_labeled_cycles_df = (
    reduce(lambda _df, ic: _df.withColumn(
        '{0}_avg'.format(ic), F.avg(ic).over(w)), input_columns, labeled_cycles_df)
    .withColumn('is_valid', F.count(F.lit(1)).over(w) > lookback)
    .where(F.col('is_valid'))
    .drop('is_valid'))

augmented_labeled_cycles_df.printSchema()

## Persisting the feature data set

For the sake of generality, all domain-specific feature names will be obfuscated, and explicit timestamps will be replaced with numeric entry IDs indicating the relative temporal order.

In [ ]:
feature_columns = [
    c for c in augmented_labeled_cycles_df.columns if c not in (
        'machineID', 'cycle', 'sequenceID', 'rul', 'upcoming_failure', 'immediate_failure', 'timestamp')]
obfuscate_columns = zip(feature_columns, range(1, len(feature_columns) + 1))

feature_df = reduce(lambda _df, ic: _df.withColumnRenamed(ic[0], 's{0}'.format(ic[1])),
    obfuscate_columns, augmented_labeled_cycles_df).withColumn(
    'entryID', F.row_number().over(Window().orderBy('timestamp'))
).drop('upcoming_failure', 'timestamp')

feature_df.printSchema()

In [ ]:
chunks = 5 # split the data into N chunks
feature_df.coalesce(chunks).write.csv(data_dir + '/features',mode="overwrite", header=True)

## References
https://www.youtube.com/watch?v=drUToKxEAUA

http://waset.org/publications/10006640/building-a-scalable-telemetry-based-multiclass-predictive-maintenance-model-in-r

http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.386.8108&rep=rep1&type=pdf